In [1]:
from threaded_sweep import Sweep1D, Sweep0D, Sweep2D
from daq_driver import Daq, DaqAOChannel, DaqAIChannel
import nidaqmx
import time
import numpy as np
import qcodes as qc
from tracking import track_setpoint
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.database import initialise_or_create_database_at
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

In [ ]:
# initialize the lakeshore

ls = Model_372('lakeshore_372', 'TCPIP::10.155.95.127::7777::SOCKET')
ls.ch01.enabled(False)
ls.ch02.enabled(False)
ls.ch03.enabled(False)
ls.ch05.enabled(False)
ls.ch06.enabled(False)
for ch in ls.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

In [ ]:
# initialize the AMI

from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
instrument = AMI430("testmagnet", address="10.155.94.127", port=7180, has_current_rating=False)

In [2]:
# initialize the DAQ
daq = Daq("Dev1", "testdaq")

In [3]:
daq.ai0.voltage.label="I"
daq.ai1.voltage.label="Vxx"
#daq.ai2.voltage.label="Vxy"
#params = (daq.ai0.voltage)
%matplotlib qt

In [ ]:
s = Sweep1D(daq.ao0.voltage, 4, 2, 0.05, inter_delay = 0.2, bidirectional=True, plot_data=False, save_data=False)

In [5]:
# testing 2d sweep
# Set the sweep parameters -
# [ Parameter, start point, end point, step ]
in_params = [daq.ao0.voltage, 0, 0.5, 0.01]
out_params = [daq.ao1.voltage, 0, 1, 0.1]
s2d = Sweep2D(in_params, out_params, inter_delay = 0.05, save_data=False)
s2d.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage)
s2d.set_update_rule(lambda s, l: track_setpoint(s, l, 0.250, steps=15))
s2d._create_measurement()


In [6]:
%matplotlib qt
s2d.start()

Starting the 2D Sweep. Ramping ao1 Voltage to 1 V, while sweeping ao0 Voltage between 0 V and 0.5 V
Ramping to our starting setpoint value of 0 V
Done ramping ao0 Voltage to 0
Sweeping ao0 Voltage to 0.5 V
Done with the sweep, ao0 Voltage=-8.673617379884035e-17
called heatmap from thread: <sip.voidptr object at 0x0000013F15EA7A80>
Setting ao1 Voltage to 0.1 V
Ramping to our starting setpoint value of 0.175 V
Sweeping ao0 Voltage to 0.175 V
Ramping ao0 Voltage to 0.175 . . . 
Done with the sweep, ao0 Voltage=0.17999999999999994
Done ramping ao0 Voltage to 0.175
Sweeping ao0 Voltage to 0.32499999999999996 V
Done with the sweep, ao0 Voltage=0.175
called heatmap from thread: <sip.voidptr object at 0x0000013F1889AF30>
Setting ao1 Voltage to 0.2 V
Ramping to our starting setpoint value of 0.18000000000000005 V
Sweeping ao0 Voltage to 0.18000000000000005 V
Ramping ao0 Voltage to 0.18000000000000005 . . . 
Done with the sweep, ao0 Voltage=0.185
Done ramping ao0 Voltage to 0.18000000000000005
S

In [ ]:
s2d.is_running


In [ ]:
s2d.heatmap_plotter.heatmap_data[6]

In [ ]:
s2d.stop()

In [ ]:
s2d.heatmap_plotter.in_keys

In [ ]:
s2d.heatmap_plotter.count

In [ ]:
s = Sweep0D(save_data=False)

In [ ]:
s.follow_param(daq.ai0.voltage)

In [ ]:
s.start()

In [ ]:
s.stop()

In [ ]:
s.end


In [ ]:
s.begin

In [ ]:
from PyQt5.QtCore import QThread
QThread.currentThreadId()